<a href="https://colab.research.google.com/github/havihaviplants/Machine-learning/blob/main/1204.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [ ]:
# 자신의 Google Drive 마운트하는 코드를 추가하자!
mnist = np.load('/content/drive/MyDrive/MY Drive/mnist.npz')
x_train = (mnist['x_train'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_train = mnist['y_train']
x_test = (mnist['x_test'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_test = mnist['y_test']
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
# Train Data Set
train_dataset = TensorDataset(torch.FloatTensor(x_train), torch.LongTensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# Test Data Set
test_dataset = TensorDataset(torch.FloatTensor(x_test), torch.LongTensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
dimension = 28
sequence_length = 28
hidden_size = 128
num_layers = 2
num_classes = 10


In [ ]:
class RNN(nn.Module):
 def __init__(self, dimension, hidden_size, num_layers, num_classes, device):
  super().__init__()

  self.dimension = dimension
  self.hidden_size = hidden_size
  self.num_layers = num_layers
  self.num_classes = num_classes
  self.device = device

 # if batch_fist == True, then x is (batch_size, sequence_length, dimension)
 # otherwise, x is (sequence_length, batch_size, dimension)
  self.lstm = nn.LSTM(self.dimension, self.hidden_size, self.num_layers, batch_first=True)
  self.fc = nn.Linear(self.hidden_size, self.num_classes)

  self.attention = torch.nn.MultiheadAttention(embed_dim=self.hidden_size, num_heads=4, batch_first=True)

 def forward(self, x):
 # (num_layers, batch_size, hidden_size)
  h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
  c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

  x, _ = self.lstm(x, (h0, c0))
  x, _ = self.attention(x[:, -1, :], x[:, -1, :], x[:, -1, :])
  x = self.fc(x)

  return x


In [ ]:
from torch.optim import Adam
model = RNN(dimension=dimension, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes, device=device).to(device)
criterion = nn.CrossEntropyLoss().to(device)
opti = Adam(model.parameters(), lr=1e-4)

In [ ]:
def train(model, dataloader, criterion, data_len, opti):
 correct = 0

 model.train()
 for data, target in dataloader:
  data = data.view(-1, sequence_length, dimension).to(device)
 target = target.to(device)

 output = model(data)
 loss = criterion(output, target)

 opti.zero_grad()
 loss.backward()
 opti.step()

 pred = output.max(1, keepdim=True)[1]
 correct += pred.eq(target.view_as(pred)).sum().item()

 acc = 100. * correct / data_len
 return acc


In [ ]:
def evaluate(model, dataloader, criterion, data_len):
 correct = 0

 model.eval()
 for data, target in dataloader:
  data = data.view(-1, sequence_length, dimension).to(device)
 target = target.to(device)

 output = model(data)
 loss = criterion(output, target)

 pred = output.max(1, keepdim=True)[1]
 correct += pred.eq(target.view_as(pred)).sum().item()

 acc = 100. * correct / data_len
 return acc

In [ ]:
epoch = 10
for i in range(epoch):
 train_acc = train(model, train_dataloader, criterion, len(train_dataloader.dataset), opti)
 val_acc = evaluate(model, test_dataloader, criterion, len(test_dataloader.dataset))
 print(f"[Epoch: {i:2d}], [Train Acc: {train_acc:3.4f}], [Val Acc: {val_acc:3.4f}]")

[Epoch:  0], [Train Acc: 0.0050], [Val Acc: 0.0200]
[Epoch:  1], [Train Acc: 0.0067], [Val Acc: 0.0200]
[Epoch:  2], [Train Acc: 0.0033], [Val Acc: 0.0200]
[Epoch:  3], [Train Acc: 0.0017], [Val Acc: 0.0200]
